In [11]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
import scipy as sp

import tensorflow as tf

%matplotlib inline
%precision 4
plt.style.use('ggplot')

# DD2447: Statistical Methods in Applied Computer Science 

## Assignemnt 2

We start by generating the data:

In [2]:
import SVDataGenerator

# T is the number of timesteps we want to generate
T = 500

# generation of the parameters phi,alpha,beta
parameters = SVDataGenerator.sv_parameter_generator(saveOpt=True, displayOpt=True, filename='myparameters.csv')
# generation of a sequence of 500 log-returns
yt = SVDataGenerator.sv_data_generator(parameters, T, displayOpt=True, saveOpt=True, filename='mydata.csv')

Generated parameters:
 phi =  0.965825273948
 sigma =  0.178540302347
 beta =  0.147271199563
Saved to  myparameters.csv
Data generated with parameters: (0.96582527394785755, 0.17854030234745533, 0.14727119956264476)
Saved to  mydata.csv


In [3]:
yt = np.array(yt)
phi    = parameters[0]
sigma  = parameters[1]
beta   = parameters[2]

In [4]:
class ProbabilityFunction:
    
    def __init__(self):
        pass
    
    def rvs(self):
        pass
    
    def pdf(self,x):
        pass
    
    def cdf(self,x):
        pass

In [5]:
class MultipleUniformProposals(ProbabilityFunction):
    
    def __init__(self,f_list):
        self.f_list=f_list
        
    

### Task 2.2 MCMC for the stohastic volatiliry model

We will now describe an MCMC sampler for the stohastic volatility model.

1. A time index t is fist sampled uniformly in $\{0, ... , T\}$.

2. Sample $x_t$ from the conditional distribution $p(x_t \mid \theta, x_{0:t-1}, x_{t+1:T},y_{1:T})$, which is given, up to a normalizing constant, by the likelihood function

$ p(x_{0:T}, y_{1:T} \mid \theta) = $

$ \mathcal{N}( x_0 \mid 0, \frac{\sigma^2}{1- \phi^2}) \sum_{t=1}^T [ \mathcal{N}(x_t \mid \phi x_{t-1},\sigma^2 ) \mathcal{N}(y_t \mid 0, \beta^2 exp(x_t))] 
$

In order to sample $x_t$, use a Metropolis-Hastings sampler characterized by one of the followin proposal samling densities $g_t(x_t' \mid x_t)$ each chosen with probability $1/3$:

1. If $t>0$, sample $ v_n \sim \mathcal{N}(v_n \mid 0,1)$ and set $ x_t' = \phi x_{t-1} + \sigma v_n $ . Otherwise, sample $ x_0' \sim \mathcal{N}(x_0' \mid 0, \frac{\sigma^2}{1- \phi^2})$ .
2. If $t<T$, sample $ v_n \sim \mathcal{N}(v_n \mid 0,1)$ and set $ x_t' = \frac{x_{t+1} - \sigma v_n}{\phi} $ . Otherwise, set $ x_T' = x_T$ .
3. If $t>0$, sample $ w_n \sim \mathcal{N}(w_n \mid 0,1)$ and set $ x_t' = 2 log(\frac{|y_t|}{\beta |w_n|}) $ . Otherwise, set $ x_0' = x_0$ .



#### Answer:

In this task we use Markov Chain Monte Carlo (MCMC) method to generate samples for the distribution $p(x_{0:T}, y_{1:T} \mid \theta)$. In general, the method starts by constucting a Markov chain whose stationary distribution is the target density of interest. 

MCMC method of choice is the Metropolis Hastings algorithm.
Apart from the distribution we want to sample, the algorithm requires another proposal distribution $g(\mathbf{x'} \mid \mathbf{x})$.


Having defined the proposal distribution we start by randomly initializing the initial state $\mathbf{x_0}$. Then we sample a new timestep $ t \sim U(0,T)$ which denotes the timestep of the stohastic volatility model we sample (not the Monte Carlo timestep). 

Next we sample a new $x' \sim g(x' \mid x)$ and compute the acceptance probability $ \alpha = \frac{ \widetilde{p}(x')g(x \mid x') }{  \widetilde{p}(x)g(x' \mid x) }$. If the ratio is higher than one the next state has a higher probability than the previous one and we will always take it, therefore the new acceptance probability is set to 1. Otherwise we want to encourage exploration around the state and we consider taking the next state with the probability $\alpha$.  

##### Initialisation

The initial state $\mathbf{x_0}$ needs to have a non-zero probability. If this condition isn't met there might be problems with computing the acceptance probability, if the denominator probability is zero acceptance probability is infinite, but that could be reported as a runtime error depending on the programming language and machine we use. The other possibility is that the proposed new state is still in the zero-probability area which would cause a case of $\frac{0}{0}$. Therefore we use Tensorflow framework for function optimisation using directed graphs with automatic operation differentitation. We build the computational graph for function:

$ p(x_{0:T}, y_{1:T} \mid \theta) = $

$ \mathcal{N}( x_0 \mid 0, \frac{\sigma^2}{1- \phi^2}) \sum_{t=1}^T [ \mathcal{N}(x_t \mid \phi x_{t-1},\sigma^2 ) \mathcal{N}(y_t \mid 0, \beta^2 exp(x_t))] 
$

Initialise a random initial state $x_{0:T}$ and use a stohastic gradient descent algorithm to find the **MAP** estimation for the given data $y_{1:T}$.

In [37]:
def N(x,mu,sigma,scope):
    with tf.name_scope(scope):
        return 1. / tf.sqrt(2*np.pi*sigma**2) * tf.exp( -(x-mu)**2/(2*sigma**2) )

In [38]:
# tensorflow grap construction
g = tf.Graph()
opt_steps = 1000

with g.as_default():
    #tf.reset_default_graph()
    # build graph
    with tf.name_scope('placeholders'):
        y_ = tf.placeholder(tf.float32,[T])
        sigma_ = tf.placeholder(tf.float32)
        phi_ = tf.placeholder(tf.float32)
        beta_ = tf.placeholder(tf.float32)
        print('Setup placeholders...')

    with tf.variable_scope('variables'):
        x = tf.get_variable('x',
                            shape=[T+1],
                            trainable=True,
                            initializer=tf.constant_initializer(np.zeros(T+1)))
        print('Setup variables...')


    p_list=[]
    p = N(x[0], 0., sigma_**2/(1-phi_**2),'N0')
    p_list.append(p)
    tf.summary.scalar('p_0',p)

    for t in range(1,T+1):
        p *= N(x[t], phi_ * x[t-1],sigma_**2, 'Nx{}'.format(t))
        p *= N(y_[t-1],0., beta_**2 * tf.exp(x[t]), 'Ny{}'.format(t))
        p_list.append(p)
        tf.summary.scalar('p_{}'.format(t),p)        
        print('Chain: {}/{}'.format(t,T))
    print('Setup chain...')

    with tf.name_scope('loss'):
        loss = -tf.log(p)
        tf.summary.scalar('loss',loss)
        print('Setup loss...')

    with tf.name_scope('train'):
        train_op = tf.train.AdamOptimizer().minimize(loss)
        print('Generated training op...')   

    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./tensorboard_log',graph=g)

Setup placeholders...
Setup variables...
Chain: 1/500
Chain: 2/500
Chain: 3/500
Chain: 4/500
Chain: 5/500
Chain: 6/500
Chain: 7/500
Chain: 8/500
Chain: 9/500
Chain: 10/500
Chain: 11/500
Chain: 12/500
Chain: 13/500
Chain: 14/500
Chain: 15/500
Chain: 16/500
Chain: 17/500
Chain: 18/500
Chain: 19/500
Chain: 20/500
Chain: 21/500
Chain: 22/500
Chain: 23/500
Chain: 24/500
Chain: 25/500
Chain: 26/500
Chain: 27/500
Chain: 28/500
Chain: 29/500
Chain: 30/500
Chain: 31/500
Chain: 32/500
Chain: 33/500
Chain: 34/500
Chain: 35/500
Chain: 36/500
Chain: 37/500
Chain: 38/500
Chain: 39/500
Chain: 40/500
Chain: 41/500
Chain: 42/500
Chain: 43/500
Chain: 44/500
Chain: 45/500
Chain: 46/500
Chain: 47/500
Chain: 48/500
Chain: 49/500
Chain: 50/500
Chain: 51/500
Chain: 52/500
Chain: 53/500
Chain: 54/500
Chain: 55/500
Chain: 56/500
Chain: 57/500
Chain: 58/500
Chain: 59/500
Chain: 60/500
Chain: 61/500
Chain: 62/500
Chain: 63/500
Chain: 64/500
Chain: 65/500
Chain: 66/500
Chain: 67/500
Chain: 68/500
Chain: 69/500
Ch

KeyboardInterrupt: 

In [ ]:
with g.as_default():
    print('Starting training...')
    feed_dict={y_:yt, sigma_:sigma,phi_:phi,beta_:beta}

    # initalise session
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        x0,p0,loss0 = sess.run([x,p,loss],feed_dict={y_:yt, sigma_:sigma,phi_:phi,beta_:beta})
        print('s:{} | l: {} | p:{}'.format(s,loss_,p0))        
        for s in range(opt_steps):
            _,loss_,x_s,p_s,summary = sess.run([train_op, loss,x,p,merged],feed_dict=feed_dict)
            p_list_ = sess.run(p_list,feed_dict=feed_dict)
            writer.add_summary(summary,s)
            print('s:{} | l: {} | p: {}'.format(s,loss_,p_s))

In [32]:
x_s

array([ 0.0017,  0.0017], dtype=float32)

##### Modeling the proposal density

Since we are given three different proposal sampling densities, each having the same probability of being chosen, we can construct the following Mixture proposal. We can interpret this as marginalization of a discrete random variable $I$ over three possible classes, one for each function:

$$ g(x' \mid x) = \sum_{k=1}^3 g(x' \mid x, I=i) p(I=i) $$

where $p(I=i)=\frac{1}{3}$ for $ k \in \{ 1,2,3 \}$. 

We then write $ g(x' \mid x, I=i) $ as :

$$ g(x' \mid x, I=i) = g^1(x' \mid x)  \mathbf{1}_{i==1} + g^2(x' \mid x)  \mathbf{1}_{i==2} + g^3(x' \mid x)  \mathbf{1}_{i==3}  $$

where $\mathbf{1}_{L}$ is the indicator function witch results in $1$ if $L$ is true, and in $0$ otherwise. 

Each of $g^i(x' \mid x)$ is a probability density function for the next state $x'$. They are defined as functions of model parameters $\Theta = \{ \sigma, \phi, \beta \}$, predictions $y_{1:T}$, latent variables $x_{0:T}$ and random variables $v_n$ and $w_n$. Therefore, in order to sample from the proposal distribution it is necessary to use the change-of-variable rule for computing the right probability density function and cumulative distribution function needed for the inverse probability transform method of sampling. 

We start with the pdf $g^1(x' \mid x)$. If the selected timestep $ t \sim U(0,T)$ turns out to be $0$ we sample from the normal distribution $\mathcal{N}(x_0' \mid 0, \frac{\sigma^2}{1-\phi^2})$. Otherwise we obtain $x_t'$ as:

$$ x_t' = \phi x_{t-1} + \sigma v_n , \quad v_n \sim \mathcal{N}(v_n \mid 0,1) $$

The change-of-variable rule is defined as:

$$ f_Y(y) = \left | \frac{d}{dy} h^{-1}(y) \right | f_X(h^{-1}(y)) \quad \forall y \in g(X)$$

Inverse transformation is:

$$ v_n(x_t') = \frac{1}{\sigma} ( x_t' - \phi x_{t-1})$$

The derivative is :

$$ \frac{d}{d x_t'} v_n(x_t') = \frac{1}{\sigma} $$

And with:

$$  f_{V_n}( x ) = \sqrt{\frac{2}{\pi}} exp\{ -\frac{x^2}{2} \} $$

We obtain:

$$ f_{X_t'}(x_t') = \left | \frac{d}{dx_t'} v_n(x_t') \right | f_{V_n}(v_n(x_t')) $$

$$ f_{X_t'}(x_t') = \frac{1}{\sigma} \sqrt{\frac{2}{\pi}} exp\{ -\frac{(\frac{1}{\sigma} ( x_t' - \phi x_{t-1}))^2}{2} \} $$

$$ f_{X_t'}(x_t') = \frac{1}{\sqrt{2 \pi \sigma}} exp\{ -\frac{( x_t' - \phi x_{t-1})^2}{2 \sigma^2} \} $$

$$ f_{X_t'}(x_t') = \mathcal{N}(x_t' \mid \phi x_{t-1}, \sigma^2) $$

As we have proven, the transformed pdf is also a normal distribution with just a different set of parameters. Rewriting to original notation:

$$ g^1(x' \mid x) = \mathcal{N}(x_t' \mid \phi x_{t-1}, \sigma^2) $$

Putting it all together:

\begin{equation}
    g^1(x' \mid x)=
    \begin{cases}
      \mathcal{N}(x_0' \mid 0, \frac{\sigma^2}{1-\phi^2}), & \text{if}\ t=0 \\
      \mathcal{N}(x_t' \mid \phi x_{t-1}, \sigma^2) , & \text{otherwise}
    \end{cases}
  \end{equation}

$$ f_{X_t'}(x_t') =  \frac{1}{ \sqrt{2 \pi \frac{\sigma^2}{\phi^2} }} exp\{ -\frac{ (\frac{x_{t+1} - \phi x_t'}{\sigma})^2}{2} \}$$

$$ f_{X_t'}(x_t') =  \frac{1}{ \sqrt{2 \pi \frac{\sigma^2}{\phi^2} }} exp\{ -\frac{ ( x_t' - \frac{x_{t+1}}{\phi} )^2}{2 \frac{\phi^2}{\sigma^2}} \}$$

$$ f_{X_t'}(x_t') = \mathcal{N}(x_t' \mid \frac{x_{t+1}}{\phi}, \frac{\phi^2}{\sigma^2}) $$


$$ g^2(x' \mid x) = \mathcal{N}(x_t' \mid \frac{x_{t+1}}{\phi}, \frac{\phi^2}{\sigma^2}) $$



Then, the second case also consideres two different distribution with respect to $t \sim U(0,T)$. If the $t = T$ then we just copy the last value to the next: $x_T' = x_T$ .
Analogous to the first case, the second case will have: 

$$ x_t' =  \frac{( x_{t+1} + \sigma v_n)}{\phi}  , \quad v_n \sim \mathcal{N}(v_n \mid 0,1) $$

Inverse transformation is:

$$ v_n(x_t') = \frac{x_{t+1} - \phi x_t'}{\sigma}  $$

The derivative is :

$$ \frac{d}{d x_t'} v_n(x_t') = -\frac{\phi}{\sigma} $$

And with:

$$  f_{V_n}( x ) = \sqrt{\frac{2}{\pi}} exp\{ -\frac{x^2}{2} \} $$

We obtain:

$$ f_{X_t'}(x_t') = \left | \frac{d}{dx_t'} v_n(x_t') \right | f_{V_n}(v_n(x_t')) $$

$$ f_{X_t'}(x_t') = \left | -\frac{\phi}{\sigma} \right | \sqrt{\frac{2}{\pi}} exp\{ -\frac{ (\frac{x_{t+1} - \phi x_t'}{\sigma})^2}{2} \}$$

$$ f_{X_t'}(x_t') =  \frac{1}{ \sqrt{2 \pi \frac{\sigma^2}{\phi^2} }} exp\{ -\frac{ (\frac{x_{t+1} - \phi x_t'}{\sigma})^2}{2} \}$$

$$ f_{X_t'}(x_t') =  \frac{1}{ \sqrt{2 \pi \frac{\sigma^2}{\phi^2} }} exp\{ -\frac{ ( x_t' - \frac{x_{t+1}}{\phi} )^2}{2 \frac{\phi^2}{\sigma^2}} \}$$

$$ f_{X_t'}(x_t') = \mathcal{N}(x_t' \mid \frac{x_{t+1}}{\phi}, \frac{\phi^2}{\sigma^2}) $$


$$ g^2(x' \mid x) = \mathcal{N}(x_t' \mid \frac{x_{t+1}}{\phi}, \frac{\phi^2}{\sigma^2}) $$



\begin{equation}
    g^2(x' \mid x)=
    \begin{cases}
      \mathcal{N}(x_t' \mid \frac{x_{t+1}}{\phi}, \frac{\phi^2}{\sigma^2}), & \text{if}\ t<T \\
      x_T , & \text{otherwise}
    \end{cases}
  \end{equation}

Finally for the third function we have the fllowing rules. If the sampled $t \sim U(0,T)$ is equal to $0$ we set $x_0' = x_0$. Otherwise we have the following function:

$$ x_t' =  2 log(\frac{|y_t|}{\beta |w_n|})  , \quad w_n \sim \mathcal{N}(w_n \mid 0,1) $$

In [ ]:
x = np.linspace(-10,10)
plt.plot(x, 2 * np.log( 0.01/(1*np.abs(x))));

Inverse transformation is:

$$ w_n(x_t') = \left| \frac{|y_t|}{\beta} exp \{ - \frac{x_t'}{2}\} \right | $$

In [ ]:
x = np.linspace(-10,10)
plt.plot(x, np.abs( 0.01/1 * np.exp(-x/2)));

Which is positive on the whole domain so we can exclude the absolute function ( $y_t$ is already required to be absolute and $beta>0$ by definition).


The derivative is :

$$ \frac{d}{d x_t'} w_n(x_t') = -\frac{y_t}{2\beta} exp \{ - \frac{x_t'}{2}\} $$

And with:

$$  f_{|W_n|}( x ) = \frac{2}{\sqrt{2 \pi}}  exp\{ -\frac{x^2}{2} \} \quad \forall x \in [0,\inf)$$

Note that here our interval is restricted to only positive numbers, so therefore we need to correct for it in the pdf by multiplying the current value with $2$.

We obtain:

$$ f_{X_t'}(x_t') = \left | \frac{d}{dx_t'} w_n(x_t') \right | f_{W_n}(w_n(x_t')) $$

$$ f_{X_t'}(x_t') = \left | -\frac{y_t}{2\beta} exp \{ - \frac{x_t'}{2}\} \right | \frac{2}{\sqrt{2 \pi}}  exp\{ -\frac{(\frac{y_t}{\beta} exp \{ - \frac{x_t'}{2}\})^2}{2} \} $$

$$ f_{X_t'}(x_t') = \frac{|y_t|}{\beta \sqrt{2 \pi}} exp \bigg\{ -\frac{|y_t|^2}{2 \beta^2} exp\{-x_t' \} - \frac{x_t'}{2} \bigg\} $$


\begin{equation}
    g^3(x' \mid x)=
    \begin{cases}
      x_0, & \text{if}\ t=0 \\
      \frac{|y_t|}{\beta \sqrt{2 \pi}} exp \bigg\{ -\frac{|y_t|^2}{2 \beta^2} exp\{-x_t' \} - \frac{x_t'}{2} \bigg\}, & \text{otherwise}
    \end{cases}
  \end{equation}

All relevant equations, side by side :

$$ g(x' \mid x) = \sum_{k=1}^3 g(x' \mid x, I=i) p(I=i) $$

---

$$ g(x' \mid x, I=i) = g^1(x' \mid x)  \mathbf{1}_{i==1} + g^2(x' \mid x)  \mathbf{1}_{i==2} + g^3(x' \mid x)  \mathbf{1}_{i==3}  $$

---

\begin{equation}
    g^1(x' \mid x)=
    \begin{cases}
      \mathcal{N}(x_0' \mid 0, \frac{\sigma^2}{1-\phi^2}), & \text{if}\ t=0 \\
      \mathcal{N}(x_t' \mid \phi x_{t-1}, \sigma^2) , & \text{otherwise}
    \end{cases}
  \end{equation}

---

\begin{equation}
    g^2(x' \mid x)=
    \begin{cases}
      \mathcal{N}(x_t' \mid \frac{x_{t+1}}{\phi}, \frac{\phi^2}{\sigma^2}), & \text{if}\ t<T \\
      x_T , & \text{otherwise}
    \end{cases}
  \end{equation}

---

\begin{equation}
    g^3(x' \mid x)=
    \begin{cases}
      x_0, & \text{if}\ t=0 \\
      \frac{|y_t|}{\beta \sqrt{2 \pi}} exp \bigg\{ -\frac{|y_t|^2}{2 \beta^2} exp\{-x_t' \} - \frac{x_t'}{2} \bigg\}, & \text{otherwise}
    \end{cases}
  \end{equation}


##### Acceptance probability

Acceptance probability is given by the ratio:

$$ \alpha = \frac{ \widetilde{p}(x')g(x \mid x') }{  \widetilde{p}(x)g(x' \mid x) }$$

which can be simplified to :

$$ \alpha = \frac{ \widetilde{p}(x')}{  \widetilde{p}(x) }$$

if the proposial distribution is symmetrical. If it is not symetrical we need to maintain detailed balance (reversibility) of the stationary distribution by including the ratio between proposial distributon probabilies of the previous state and the next state. 

We know that there normal distributions are always symettric so we only check for the last equation.

By definition, a probability distribution is said to be symmetrix if and only if there exists a value $x_0$ such that 
$$ f(x_0 - \delta) = f(x_0 + \delta) \quad \forall \delta \in I\!R$$

From there with $\frac{|y_t|}{\sqrt{2 \pi \beta^2}} = C_1$ and $ \frac{|y_t|^2}{2 \beta^2} = C_2 $:

$$ C_1 * exp \bigg\{ -C_2 * exp\{-(x_0 - \delta) \} - \frac{x_0 - \delta}{2} \bigg\}  = C_1 * exp \bigg\{ -C_2 * exp\{-(x_0 + \delta) \} - \frac{x_0 + \delta}{2} \bigg\}$$

$$ -C_2 * exp\{-(x_0 - \delta) \} - \frac{x_0 - \delta}{2} = -C_2 * exp\{-(x_0 + \delta) \} - \frac{x_0 + \delta}{2} $$


$$ -C_2 * exp\{-(x_0 - \delta) \} + C_2 * exp\{-(x_0 + \delta) \} =  - \frac{x_0 + \delta}{2} + \frac{x_0 - \delta}{2}  $$

$$ C_2 *  e^{-x_0} ( e^{ \delta} - e^{ -\delta } ) = \delta $$


$$ e^{-x_0}  =  \frac{\delta}{C_2( e^{ \delta} - e^{ -\delta } )} $$
$$ x_0 =  - log(\frac{\delta}{C_2( e^{ \delta} - e^{ -\delta } )} ) $$
$$ x_0 =  log(\frac{C_2( e^{\delta } - e^{ \delta}   )}{ \delta} ) $$

$$ x_0 =  log\Big( \frac{|y_t|^2}{2 \beta^2} \frac{( e^{ \delta} - e^{ -\delta }   )}{ \delta} \Big) $$

The logarithmic function is defined only for non-negative number, therefore:

$$ \frac{( e^{ \delta} - e^{ -\delta }   )}{ \delta} > 0 $$ 

$$ ( e^{ \delta} - e^{ -\delta } > 0 \land \delta >0 ) \lor ( e^{ \delta} - e^{ -\delta } < 0 \land \delta <0 )$$


In the first case, if $\delta >0$ then $ e^\delta > e^{-\delta} $ is always true. Analogous is true for the second equation, if the $\delta <0$ then $ e^\delta < e^{-\delta} $ and therefore we can conclude that the given probability densityfunction is symmetric.

In [ ]:
x=np.linspace(-5,5,1000)
plt.plot(x, np.log((np.exp(x) - np.exp(-x))/x));

$$\frac{|y_t|}{\beta \sqrt{2 \pi}} exp \bigg\{ -\frac{|y_t|^2}{2 \beta^2} exp\{-x_t' \} - \frac{x_t'}{2} \bigg\}$$

In [ ]:
def f(x_t, y_t, beta):
    return np.abs(y_t) / (2*beta*np.sqrt( 2 * np.pi )) * np.exp( -y_t**2/(2*beta**2) * np.exp(-x) - x/2 )

x= np.linspace(-100,100)
for y_t in[0.1,0.01,0.001]:
    plt.plot(x, f(x,y_t,1),label='$y_t = {}$'.format(y_t))
plt.legend();

This is why we will use the simplified acceptance probability ratio:

$$ \alpha = \frac{ \widetilde{p}(x')}{  \widetilde{p}(x) }$$

---

#### Algorithm

In [4]:
class ProbabilityFunction:
    
    def __init__(self):
        pass
    
    def rvs(self):
        pass
    
    def pdf(self,x):
        pass
    
    def cdf(self,x):
        pass

In [10]:
class TimeDepProbabilityFunction:
    
    def __init__(self,x0):
        self.x=x0

    
    def rvs(self,t):
        pass
    
    def pdf(self,t,x_t):
        pass
    
    def cdf(self,t,x_t):
        pass
    
    def update_state(self,x):
        self.x=x

$$ g(x' \mid x) = \sum_{k=1}^3 g(x' \mid x, I=i) p(I=i) $$

---

$$ g(x' \mid x, I=i) = g^1(x' \mid x)  \mathbf{1}_{i==1} + g^2(x' \mid x)  \mathbf{1}_{i==2} + g^3(x' \mid x)  \mathbf{1}_{i==3}  $$

---

In [5]:
class MultipleUniformProposals(TimeDepProbabilityFunction):
    
    def __init__(self,f_list):
        self.f_list=f_list
        self.n = len(self.f_list)
        
    def rvs(self,t):
        pass
    
    def pdf(self,t,x_t):
        pass
    
    def cdf(self,t,x_t):
        pass

---

\begin{equation}
    g^1(x' \mid x)=
    \begin{cases}
      \mathcal{N}(x_0' \mid 0, \frac{\sigma^2}{1-\phi^2}), & \text{if}\ t=0 \\
      \mathcal{N}(x_t' \mid \phi x_{t-1}, \sigma^2) , & \text{otherwise}
    \end{cases}
  \end{equation}



In [ ]:
class Proposal_1(TimeDepProbabilityFunction):

    def __init__(self,x0,phi,sigma):
        TimeDepProbabilityFunction.__init__(self, x0)
        self.phi   = phi
        self.sigma = sigma
    
    def rvs(self,t):
        if t is 0:
            return st.norm.rvs(loc=0, scale=self.sigma**2/(1-self.phi**2))
        else:
            return st.norm.rvs(loc=self.phi * self.x[t-1], scale=self.sigma**2)
        
    
    def pdf(self,t,x_t):
        if t is 0:
            return st.norm.pdf(x_t,loc=0, scale=self.sigma**2/(1-self.phi**2))
        else:
            return st.norm.pdf(x_t,loc=self.phi * self.x[t-1], scale=self.sigma**2)

    
    def cdf(self,t,x_t):
        if t is 0:
            return st.norm.cdf(x_t,loc=0, scale=self.sigma**2/(1-self.phi**2))
        else:
            return st.norm.cdf(x_t,loc=self.phi * self.x[t-1], scale=self.sigma**2)


---
\begin{equation}
    g^2(x' \mid x)=
    \begin{cases}
      \mathcal{N}(x_t' \mid \frac{x_{t+1}}{\phi}, \frac{\phi^2}{\sigma^2}), & \text{if}\ t<T \\
      x_T , & \text{otherwise}
    \end{cases}
  \end{equation}


In [ ]:
class Proposal_2(TimeDepProbabilityFunction):

    def __init__(self,x0,phi,sigma,T,eps=1e-16):
        TimeDepProbabilityFunction.__init__(self, x0)
        self.phi   = phi
        self.sigma = sigma
        self.T = T
        self.eps=eps

    
    def rvs(self,t):
        if t < T-1:
            return st.norm.rvs(loc=self.x[t+1]/(self.phi**2), scale=self.phi**2/(self.phi**2))
        else:
            return self.x[T-1]
        
    
    def pdf(self,t,x_t):
        if t < T-1:
            return st.norm.pdf(x_t,loc=self.x[t+1]/(self.phi**2), scale=self.phi**2/(self.phi**2))
        else:
            return 1. if abs(x_t-self.x[T-1]) < self.eps else 0.

    
    def cdf(self,t,x_t):
        if t < T-1:
            return st.norm.cdf(x_t,loc=x[t+1]/(self.phi**2), scale=scale=self.phi**2/(self.phi**2))
        else:
            return 0. if x_t < self.x[T-1] else 1.


---

\begin{equation}
    g^3(x' \mid x)=
    \begin{cases}
      x_0, & \text{if}\ t=0 \\
      \frac{|y_t|}{\beta \sqrt{2 \pi}} exp \bigg\{ -\frac{|y_t|^2}{2 \beta^2} exp\{-x_t' \} - \frac{x_t'}{2} \bigg\}, & \text{otherwise}
    \end{cases}
  \end{equation}

In [24]:
class Proposal_3(TimeDepProbabilityFunction):

    def __init__(self,x0,beta,y,eps=1e-6, x_start=0, d_x=10):
        TimeDepProbabilityFunction.__init__(self, x0)
        self.beta = beta
        self.y=y
        self.eps = eps
        self.x_start=0
        self.d_x=d_x
        
    def rvs(self,t):
        #if t is 0:
        #    return self.x[t]
        #else:
        #    #sample a random number
        #    t = st.uniform.rvs()
        #    # find the prob for a rando value, for example at 0 
        #    x_t_prev = x_start
        #    p_prev,_ = self.cdf(t,x_t_prev)
        #    
        #    x_t_next = x_t_prev + self.d_x
        #    p_next,_ = self.cdf(t,x_t_next)
        #    
        #    sign = +1 if 
        #    p = p_prev
        #    while abs(t-p)> eps: # until the error is less than some constant
        #        p_next 
        # Stop this madness and implement rejection sampling       
    def pdf(self,t,x_t):
        if t is 0:
            return 1. if abs(x_t-self.x[0]) < self.eps else 0.
        else:
            return self._f(t,x_t)

    def cdf(self,t,x_t):
        if t is 0:
            return 0. if x_t < self.x[0] else 1.
        else:
            return sp.integrate.quad(lambda x: self._f(t,x),np.NINF, x_t)[0] # the return is the probabiliy and error
            
    def _f(self,t,x_t):
        C = 2* np.abs(self.y[t])/(self.beta * np.sqrt(2*np.pi))
        power_1 = - self.y[t]**2/(2*self.beta**2) * np.exp(-x_t)
        power_2 = -x_t/2

        return C * np.exp(power_1 + power_2)


In [ ]:
class LikelihoodFunction(ProbabilityFunction):
    
    def __init__(self,phi,sigma,beta,yt):
        self.phi=phi
        self.sigma=sigma
        self.beta=beta
        self.yt = yt
        
    def rvs(self):
        raise Exception('LikelihoodFunction :: rvs not implemented')
        
    
    def pdf(self,x):
        p = st.norm.pdf(x[0], loc=0, scale= self.sigma**2/(1-self.phi**2))
        
        for t in range(1,len(x)) :
            p*=st.norm.pdf(x[t], loc= self.phi * x[t-1], scale= self.sigma**2)
            p*=st.norm.pdf(yt[t], loc=0, scale= self.beta**2*np.exp(x[t])
        
        return p
    
    def cdf(self,x):
        raise Exception('LikelihoodFunction :: cdf not implemented')
        

**Question 4:** Derive the Metropolis-Hastings algorithm and implement MCMC sampler. 

**Question 5:** Report the marginal distribution of $\sigma^2$ and $\beta^2$ on histograms. 